# Healthcare Business Intelligence Sample Queries

Based on the healthcare medallion architecture dimensional model, these queries evaluate key aspects of the healthcare insurance business.

## Population Health Analytics

In [ ]:
-- Query 1: Patient Risk Distribution and Premium Analysis
-- Business Question: What is the distribution of patients by risk category and how does it correlate with premium costs?
SELECT 
    p.health_risk_category,
    p.patient_age_category,
    p.patient_region,
    COUNT(*) as patient_count,
    AVG(CASE 
        WHEN p.patient_premium_category = 'LOW_PREMIUM' THEN 5000
        WHEN p.patient_premium_category = 'MEDIUM_PREMIUM' THEN 15000  
        WHEN p.patient_premium_category = 'HIGH_PREMIUM' THEN 30000
        ELSE 50000 
    END) as avg_estimated_premium,
    ROUND(AVG(p.health_risk_score), 2) as avg_risk_score,
    SUM(CASE WHEN p.patient_smoking_status = 'SMOKER' THEN 1 ELSE 0 END) as smoker_count,
    ROUND(SUM(CASE WHEN p.patient_smoking_status = 'SMOKER' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as smoking_rate_pct
FROM juan_dev.healthcare_data.dim_patients p
WHERE p.is_current_record = true
GROUP BY p.health_risk_category, p.patient_age_category, p.patient_region
ORDER BY patient_count DESC;

In [ ]:
-- Query 2: Claims Cost Analysis by Patient Risk Profile
-- Business Question: How do claim costs vary by patient risk categories and what are the cost drivers?
SELECT 
    p.health_risk_category,
    p.patient_age_category,
    COUNT(c.claim_natural_key) as total_claims,
    ROUND(SUM(c.claim_amount), 2) as total_claim_amount,
    ROUND(AVG(c.claim_amount), 2) as avg_claim_amount,
    ROUND(SUM(c.claim_amount) / COUNT(DISTINCT p.patient_natural_key), 2) as avg_cost_per_patient,
    
    -- Claim outcome analysis
    ROUND(SUM(CASE WHEN c.claim_approved = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as approval_rate_pct,
    ROUND(SUM(CASE WHEN c.claim_denied = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as denial_rate_pct,
    
    -- Processing efficiency
    ROUND(AVG(c.total_processing_days), 1) as avg_processing_days,
    
    -- High cost claims analysis
    SUM(CASE WHEN c.claim_amount > 10000 THEN 1 ELSE 0 END) as high_cost_claims,
    ROUND(SUM(CASE WHEN c.claim_amount > 10000 THEN c.claim_amount ELSE 0 END), 2) as high_cost_amount
    
FROM juan_dev.healthcare_data.dim_patients p
INNER JOIN juan_dev.healthcare_data.fact_claims c ON p.patient_surrogate_key = c.patient_surrogate_key
WHERE p.is_current_record = true
GROUP BY p.health_risk_category, p.patient_age_category
ORDER BY total_claim_amount DESC;

## Care Coordination and Utilization Analytics

In [ ]:
-- Query 3: Healthcare Utilization Patterns by Demographics
-- Business Question: How do healthcare utilization patterns vary across patient demographics?
WITH patient_utilization AS (
    SELECT 
        p.patient_natural_key,
        p.health_risk_category,
        p.patient_age_category,
        p.patient_region,
        p.demographic_segment,
        
        -- Medical events utilization
        COUNT(me.event_natural_key) as total_medical_events,
        SUM(CASE WHEN me.emergency_visit = true THEN 1 ELSE 0 END) as emergency_visits,
        SUM(CASE WHEN me.hospital_admission = true THEN 1 ELSE 0 END) as hospital_admissions,
        SUM(CASE WHEN me.preventive_care_indicator = true THEN 1 ELSE 0 END) as preventive_care_events,
        SUM(CASE WHEN me.acute_care_indicator = true THEN 1 ELSE 0 END) as acute_care_events,
        
        -- Care coordination metrics
        AVG(me.days_since_previous_event) as avg_care_gap_days,
        COUNT(DISTINCT me.medical_provider) as unique_providers,
        AVG(me.clinical_outcome_score) as avg_clinical_outcome_score
        
    FROM juan_dev.healthcare_data.dim_patients p
    LEFT JOIN juan_dev.healthcare_data.fact_medical_events me ON p.patient_surrogate_key = me.patient_surrogate_key
    WHERE p.is_current_record = true
    GROUP BY p.patient_natural_key, p.health_risk_category, p.patient_age_category, p.patient_region, p.demographic_segment
)
SELECT 
    health_risk_category,
    patient_age_category,
    patient_region,
    COUNT(*) as patient_count,
    
    -- Utilization averages
    ROUND(AVG(total_medical_events), 1) as avg_events_per_patient,
    ROUND(AVG(emergency_visits), 2) as avg_emergency_visits,
    ROUND(AVG(hospital_admissions), 2) as avg_hospital_admissions,
    
    -- Care type distribution  
    ROUND(AVG(preventive_care_events * 100.0 / NULLIF(total_medical_events, 0)), 1) as preventive_care_pct,
    ROUND(AVG(acute_care_events * 100.0 / NULLIF(total_medical_events, 0)), 1) as acute_care_pct,
    
    -- Care coordination metrics
    ROUND(AVG(avg_care_gap_days), 1) as avg_days_between_visits,
    ROUND(AVG(unique_providers), 1) as avg_providers_per_patient,
    ROUND(AVG(avg_clinical_outcome_score), 1) as avg_clinical_outcomes,
    
    -- High utilizers identification
    SUM(CASE WHEN emergency_visits >= 2 THEN 1 ELSE 0 END) as frequent_emergency_users,
    SUM(CASE WHEN total_medical_events >= 10 THEN 1 ELSE 0 END) as high_utilizers
    
FROM patient_utilization
GROUP BY health_risk_category, patient_age_category, patient_region
ORDER BY avg_events_per_patient DESC;

## Financial Performance and Cost Management

In [ ]:
-- Query 4: Monthly Financial Performance Trending
-- Business Question: What are the monthly financial trends and seasonal patterns?
SELECT 
    c.year_month,
    c.claim_quarter,
    c.patient_region,
    
    -- Volume metrics
    SUM(c.total_claims) as total_claims,
    SUM(c.unique_patients) as unique_patients,
    ROUND(SUM(c.total_claims) * 1.0 / SUM(c.unique_patients), 2) as claims_per_patient,
    
    -- Financial metrics
    SUM(c.total_claim_amount) as total_claim_amount,
    ROUND(SUM(c.total_claim_amount) / SUM(c.total_claims), 2) as avg_claim_amount,
    ROUND(SUM(c.total_claim_amount) / SUM(c.unique_patients), 2) as avg_cost_per_patient,
    
    -- Outcome metrics
    ROUND(SUM(c.approved_claims) * 100.0 / SUM(c.total_claims), 2) as approval_rate_pct,
    ROUND(SUM(c.denied_claims) * 100.0 / SUM(c.total_claims), 2) as denial_rate_pct,
    ROUND(SUM(c.paid_claims) * 100.0 / SUM(c.total_claims), 2) as payment_rate_pct,
    
    -- Efficiency metrics
    ROUND(AVG(c.avg_processing_days), 1) as avg_processing_days,
    ROUND(AVG(c.avg_data_quality_score), 1) as avg_data_quality_score,
    
    -- Year-over-year comparison (if multiple years available)
    LAG(SUM(c.total_claim_amount), 12) OVER (PARTITION BY c.patient_region ORDER BY c.year_month) as prior_year_amount,
    ROUND((SUM(c.total_claim_amount) - LAG(SUM(c.total_claim_amount), 12) OVER (PARTITION BY c.patient_region ORDER BY c.year_month)) * 100.0 / 
          NULLIF(LAG(SUM(c.total_claim_amount), 12) OVER (PARTITION BY c.patient_region ORDER BY c.year_month), 0), 2) as yoy_growth_pct
    
FROM juan_dev.healthcare_data.fact_claims_monthly_summary c
GROUP BY c.year_month, c.claim_quarter, c.patient_region
ORDER BY c.year_month, c.patient_region;

## Provider Network and Quality Analytics

In [ ]:
-- Query 5: Provider Performance and Network Analysis
-- Business Question: Which providers deliver the best clinical outcomes and cost efficiency?
WITH provider_metrics AS (
    SELECT 
        me.medical_provider,
        me.provider_type,
        me.facility_type,
        
        -- Volume metrics
        COUNT(*) as total_events,
        COUNT(DISTINCT me.patient_natural_key) as unique_patients,
        
        -- Clinical quality metrics
        AVG(me.clinical_outcome_score) as avg_clinical_outcome_score,
        AVG(me.care_efficiency_score) as avg_care_efficiency_score,
        AVG(me.care_appropriateness_score) as avg_care_appropriateness_score,
        
        -- Patient experience metrics
        AVG(me.visit_duration_minutes) as avg_visit_duration,
        SUM(CASE WHEN me.follow_up_required = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as follow_up_rate_pct,
        SUM(CASE WHEN me.emergency_visit = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as emergency_rate_pct,
        
        -- Care coordination
        AVG(me.provider_patient_familiarity) as avg_patient_familiarity,
        SUM(CASE WHEN me.is_new_provider = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as new_patient_rate_pct,
        
        -- Risk mix (case complexity)
        AVG(CASE 
            WHEN p.health_risk_category = 'LOW_RISK' THEN 1
            WHEN p.health_risk_category = 'MODERATE_RISK' THEN 2  
            WHEN p.health_risk_category = 'HIGH_RISK' THEN 3
            ELSE 4 
        END) as avg_patient_risk_score
        
    FROM juan_dev.healthcare_data.fact_medical_events me
    INNER JOIN juan_dev.healthcare_data.dim_patients p ON me.patient_surrogate_key = p.patient_surrogate_key
    WHERE p.is_current_record = true
    GROUP BY me.medical_provider, me.provider_type, me.facility_type
    HAVING COUNT(*) >= 10  -- Minimum volume for statistical significance
)
SELECT 
    *,
    -- Performance scoring (higher is better)
    ROUND((avg_clinical_outcome_score + avg_care_efficiency_score + avg_care_appropriateness_score) / 3, 2) as overall_performance_score,
    
    -- Risk-adjusted performance
    ROUND(avg_clinical_outcome_score / avg_patient_risk_score, 2) as risk_adjusted_clinical_score,
    
    -- Provider tier classification
    CASE 
        WHEN avg_clinical_outcome_score >= 40 AND avg_care_efficiency_score >= 80 THEN 'TOP_TIER'
        WHEN avg_clinical_outcome_score >= 25 AND avg_care_efficiency_score >= 60 THEN 'HIGH_PERFORMING'  
        WHEN avg_clinical_outcome_score >= 15 THEN 'STANDARD'
        ELSE 'NEEDS_IMPROVEMENT'
    END as provider_tier
    
FROM provider_metrics
WHERE total_events >= 20  -- Focus on providers with meaningful volume
ORDER BY overall_performance_score DESC;

## HIPAA Compliance and Data Quality Monitoring

In [ ]:
-- Query 6: HIPAA Compliance and Data Quality Dashboard
-- Business Question: Are we maintaining HIPAA compliance and data quality standards?
SELECT 
    'Patient Demographics' as data_domain,
    COUNT(*) as total_records,
    
    -- HIPAA compliance metrics
    SUM(CASE WHEN hipaa_deidentification_applied = true THEN 1 ELSE 0 END) as hipaa_compliant_records,
    ROUND(SUM(CASE WHEN hipaa_deidentification_applied = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as hipaa_compliance_rate_pct,
    
    SUM(CASE WHEN age_privacy_protection = true THEN 1 ELSE 0 END) as age_protected_records,
    SUM(CASE WHEN geographic_privacy_protection = true THEN 1 ELSE 0 END) as geo_protected_records,
    
    -- Data quality metrics  
    ROUND(AVG(patient_data_quality_score), 2) as avg_data_quality_score,
    SUM(CASE WHEN patient_data_quality_score >= 99.5 THEN 1 ELSE 0 END) as high_quality_records,
    ROUND(SUM(CASE WHEN patient_data_quality_score >= 99.5 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as quality_sla_compliance_pct,
    
    -- Data retention compliance
    SUM(CASE WHEN data_retention_compliance = true THEN 1 ELSE 0 END) as retention_compliant_records,
    
    -- Record quality distribution
    SUM(CASE WHEN patient_record_quality = 'EXCELLENT' THEN 1 ELSE 0 END) as excellent_quality,
    SUM(CASE WHEN patient_record_quality = 'GOOD' THEN 1 ELSE 0 END) as good_quality,
    SUM(CASE WHEN patient_record_quality = 'FAIR' THEN 1 ELSE 0 END) as fair_quality,
    SUM(CASE WHEN patient_record_quality = 'POOR' THEN 1 ELSE 0 END) as poor_quality
    
FROM juan_dev.healthcare_data.dim_patients
WHERE is_current_record = true

UNION ALL

-- Claims data quality
SELECT 
    'Insurance Claims' as data_domain,
    COUNT(*) as total_records,
    COUNT(*) as hipaa_compliant_records,  -- All claims are HIPAA processed
    100.0 as hipaa_compliance_rate_pct,
    0 as age_protected_records,  -- N/A for claims
    0 as geo_protected_records,  -- N/A for claims  
    ROUND(AVG(claim_data_quality_score), 2) as avg_data_quality_score,
    SUM(CASE WHEN claim_data_quality_score >= 99.5 THEN 1 ELSE 0 END) as high_quality_records,
    ROUND(SUM(CASE WHEN claim_data_quality_score >= 99.5 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as quality_sla_compliance_pct,
    COUNT(*) as retention_compliant_records,  -- All claims retained per policy
    0 as excellent_quality,  -- Claims use different quality scoring
    0 as good_quality,
    0 as fair_quality,
    0 as poor_quality
FROM juan_dev.healthcare_data.fact_claims

UNION ALL

-- Medical events data quality  
SELECT 
    'Medical Events' as data_domain,
    COUNT(*) as total_records,
    COUNT(*) as hipaa_compliant_records,  -- All events are HIPAA processed
    100.0 as hipaa_compliance_rate_pct,
    0 as age_protected_records,  -- N/A for events
    0 as geo_protected_records,  -- N/A for events
    ROUND(AVG(event_data_quality_score), 2) as avg_data_quality_score,
    SUM(CASE WHEN event_data_quality_score >= 99.5 THEN 1 ELSE 0 END) as high_quality_records,
    ROUND(SUM(CASE WHEN event_data_quality_score >= 99.5 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as quality_sla_compliance_pct,
    COUNT(*) as retention_compliant_records,  -- All events retained per policy
    0 as excellent_quality,  -- Events use different quality scoring  
    0 as good_quality,
    0 as fair_quality,
    0 as poor_quality
FROM juan_dev.healthcare_data.fact_medical_events;

## Predictive Analytics and Risk Stratification

In [ ]:
-- Query 7: Patient Risk Stratification for Proactive Care Management  
-- Business Question: Which patients are at highest risk for high healthcare costs and poor outcomes?
WITH patient_risk_analysis AS (
    SELECT 
        p.patient_natural_key,
        p.patient_age_category,
        p.health_risk_category,
        p.health_risk_score,
        
        -- Historical utilization (trailing 12 months)
        COUNT(me.event_natural_key) as total_medical_events_12m,
        SUM(CASE WHEN me.emergency_visit = true THEN 1 ELSE 0 END) as emergency_visits_12m,
        SUM(CASE WHEN me.hospital_admission = true THEN 1 ELSE 0 END) as admissions_12m,
        SUM(c.claim_amount) as total_claim_costs_12m,
        
        -- Clinical indicators
        AVG(me.clinical_outcome_score) as avg_clinical_outcomes,
        COUNT(DISTINCT me.medical_provider) as provider_fragmentation,
        AVG(me.days_since_previous_event) as avg_care_gaps,
        
        -- Predictive risk factors
        SUM(CASE WHEN me.chronic_management_indicator = true THEN 1 ELSE 0 END) as chronic_management_events,
        MAX(CASE WHEN me.follow_up_required = true AND me.days_to_next_event > 30 THEN 1 ELSE 0 END) as missed_followups
        
    FROM juan_dev.healthcare_data.dim_patients p
    LEFT JOIN juan_dev.healthcare_data.fact_medical_events me ON p.patient_surrogate_key = me.patient_surrogate_key
        AND me.event_date >= CURRENT_DATE - INTERVAL 12 MONTHS
    LEFT JOIN juan_dev.healthcare_data.fact_claims c ON p.patient_surrogate_key = c.patient_surrogate_key  
        AND c.claim_date >= CURRENT_DATE - INTERVAL 12 MONTHS
    WHERE p.is_current_record = true
    GROUP BY p.patient_natural_key, p.patient_age_category, p.health_risk_category, p.health_risk_score
)
SELECT 
    patient_natural_key,
    patient_age_category,
    health_risk_category,
    
    -- Composite risk scoring
    ROUND(
        (health_risk_score * 0.3) +  -- Baseline health risk
        (LEAST(emergency_visits_12m * 10, 50) * 0.2) +  -- Emergency utilization (capped)
        (LEAST(total_claim_costs_12m / 1000, 50) * 0.2) +  -- Cost utilization (scaled)
        (CASE WHEN avg_clinical_outcomes < 20 THEN 30 ELSE 0 END * 0.15) +  -- Poor outcomes
        (LEAST(provider_fragmentation * 5, 25) * 0.10) +  -- Care fragmentation  
        (CASE WHEN missed_followups = 1 THEN 20 ELSE 0 END * 0.05)  -- Care adherence
    , 2) as composite_risk_score,
    
    -- Risk stratification
    CASE 
        WHEN (health_risk_score + COALESCE(emergency_visits_12m * 10, 0) + COALESCE(total_claim_costs_12m / 1000, 0)) >= 100 THEN 'VERY_HIGH_RISK'
        WHEN (health_risk_score + COALESCE(emergency_visits_12m * 10, 0) + COALESCE(total_claim_costs_12m / 1000, 0)) >= 60 THEN 'HIGH_RISK'  
        WHEN (health_risk_score + COALESCE(emergency_visits_12m * 10, 0) + COALESCE(total_claim_costs_12m / 1000, 0)) >= 30 THEN 'MODERATE_RISK'
        ELSE 'LOW_RISK'
    END as predicted_risk_tier,
    
    -- Supporting metrics for care management
    total_medical_events_12m,
    emergency_visits_12m, 
    ROUND(COALESCE(total_claim_costs_12m, 0), 2) as total_costs_12m,
    ROUND(COALESCE(avg_clinical_outcomes, 0), 1) as avg_clinical_outcomes,
    provider_fragmentation,
    chronic_management_events,
    missed_followups,
    
    -- Care management recommendations
    CASE 
        WHEN emergency_visits_12m >= 2 THEN 'EMERGENCY_REDUCTION_PROGRAM'
        WHEN provider_fragmentation >= 5 THEN 'CARE_COORDINATION_PROGRAM'  
        WHEN chronic_management_events >= 5 THEN 'CHRONIC_DISEASE_MANAGEMENT'
        WHEN missed_followups = 1 THEN 'CARE_ADHERENCE_PROGRAM'
        ELSE 'STANDARD_CARE'
    END as recommended_intervention
    
FROM patient_risk_analysis
WHERE COALESCE(total_medical_events_12m, 0) > 0  -- Focus on patients with healthcare activity
ORDER BY composite_risk_score DESC
LIMIT 100;  -- Top 100 highest risk patients

## Executive Summary Dashboard

In [ ]:
-- Query 8: Executive KPI Summary Dashboard
-- Business Question: What are the key performance indicators for executive reporting?
WITH kpi_summary AS (
    -- Patient population metrics
    SELECT 
        'Patient Population' as metric_category,
        'Total Active Patients' as metric_name,
        COUNT(*) as metric_value,
        'count' as metric_unit
    FROM juan_dev.healthcare_data.dim_patients 
    WHERE is_current_record = true
    
    UNION ALL
    
    -- Financial metrics
    SELECT 
        'Financial Performance' as metric_category,
        'Total Claims Amount (YTD)' as metric_name, 
        ROUND(SUM(claim_amount), 0) as metric_value,
        'currency' as metric_unit
    FROM juan_dev.healthcare_data.fact_claims
    WHERE claim_year = YEAR(CURRENT_DATE)
    
    UNION ALL
    
    SELECT 
        'Financial Performance' as metric_category,
        'Average Claim Amount' as metric_name,
        ROUND(AVG(claim_amount), 0) as metric_value, 
        'currency' as metric_unit
    FROM juan_dev.healthcare_data.fact_claims
    WHERE claim_year = YEAR(CURRENT_DATE)
    
    UNION ALL
    
    -- Quality metrics
    SELECT 
        'Quality & Compliance' as metric_category,
        'Claims Approval Rate' as metric_name,
        ROUND(SUM(CASE WHEN claim_approved = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as metric_value,
        'percentage' as metric_unit
    FROM juan_dev.healthcare_data.fact_claims
    WHERE claim_year = YEAR(CURRENT_DATE)
    
    UNION ALL
    
    SELECT 
        'Quality & Compliance' as metric_category,
        'HIPAA Compliance Rate' as metric_name,
        ROUND(SUM(CASE WHEN hipaa_deidentification_applied = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as metric_value,
        'percentage' as metric_unit
    FROM juan_dev.healthcare_data.dim_patients
    WHERE is_current_record = true
    
    UNION ALL
    
    -- Utilization metrics  
    SELECT 
        'Healthcare Utilization' as metric_category,
        'Average Medical Events per Patient' as metric_name,
        ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT patient_natural_key), 2) as metric_value,
        'ratio' as metric_unit
    FROM juan_dev.healthcare_data.fact_medical_events
    WHERE event_year = YEAR(CURRENT_DATE)
    
    UNION ALL
    
    SELECT 
        'Healthcare Utilization' as metric_category, 
        'Emergency Visit Rate' as metric_name,
        ROUND(SUM(CASE WHEN emergency_visit = true THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as metric_value,
        'percentage' as metric_unit
    FROM juan_dev.healthcare_data.fact_medical_events
    WHERE event_year = YEAR(CURRENT_DATE)
)
SELECT * FROM kpi_summary
ORDER BY metric_category, metric_name;

## Notes for Business Users

### Query Execution Guidance:

1. These queries are designed to run against the gold layer dimensional model
2. Replace 'juan_dev.healthcare_data' with your actual catalog.schema names
3. Adjust date filters based on your data availability
4. Some queries use statistical minimums (e.g., minimum 10 events) for meaningful results
5. Performance can be optimized by adding appropriate indexes and partitioning

### Key Business Metrics Covered:

- **Population Health**: Risk distribution, demographics, health outcomes
- **Financial Performance**: Claims costs, approval rates, trending
- **Care Coordination**: Provider networks, utilization patterns, care gaps
- **Quality & Compliance**: HIPAA compliance, data quality, clinical outcomes
- **Predictive Analytics**: Risk stratification, intervention recommendations
- **Executive Reporting**: KPI dashboards, performance monitoring

### Dashboard Integration:

These queries can be integrated into BI tools like Tableau, Power BI, or Databricks SQL for real-time healthcare business intelligence dashboards.